In [5]:
import tweepy
import cnfg
import json
from pymongo import MongoClient

In [2]:
# first, save your config file with your key
# now access that file 

config = cnfg.load("enter path for your config file in quotes")

auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
auth.set_access_token(config["access_token"],
                      config["access_token_secret"])

api=tweepy.API(auth)

In [19]:
# we are going to store the tweets we collect in MongoDB 
# option 1: collect only the tweet elements you want, such as 'favorite_count'

client = MongoClient(port=27017)
db = client.example_live
livetweets = db.nratweets

class StreamListener(tweepy.StreamListener):

    def on_status(self, status):
        
        # skip retweets
        if status.retweeted: 
            return
    
        # store tweet and creation date
        data = {}
        data['tweet'] = status.text
        data['datetime'] = status.created_at
        #adding items of interest
        data['favorite_count'] = status.favorite_count
        data['rt_status'] = status.retweeted
        data['bio'] = status.user.description
        data['followers_count'] = status.user.followers_count
        data['location'] = status.user.location
        data['statuses_count'] = status.user.statuses_count
        data['source'] = status.source

        # insert into db
        try:
            livetweets.insert_one(data)
        except:
            pass

stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
stream.filter(track=['nra'])

ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.

In [ ]:
# option 2: collect all elements of the tweet

client = MongoClient(port=27017)
db = client.example2_live
livetweets = db.nratweets

class StreamListener(tweepy.StreamListener):

    def on_status(self, status):
        data = status._json
        
        # insert into db
        try:
            livetweets.insert_one(data)
        except:
            pass

stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener, tweet_mode=extended)
stream.filter(track=['nra'])

In [14]:
# check how many tweets we've collected so far

livetweets.count()  

72600

In [8]:
# look at one tweet

livetweets.find_one()

{'_id': ObjectId('5a94a83cc457e071515f0c54'),
 'datetime': datetime.datetime(2018, 2, 27, 0, 37, 16),
 'tweet': 'RT @JimGaffigan: The NRA’s media arm, NRATV, uses dangerous rhetoric to spread fear so people will buy more guns.\n\nAdd your name to tell @A…'}

In [18]:
livetweets[-10]

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'example_live'), 'nratweets.-10')